## Load the dataset

In [148]:
# Reading the data
import pandas as pd
import numpy as np
import csv
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy import stats
import re
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

train_data = pd.read_csv('../Datasets/preprocessed_dataset.csv', index_col=0 )
test_data = pd.read_csv('../Datasets/preprocessed_test.csv', index_col=0 )


In [149]:
train_data.head()

,Breed,Breed2,MixedBreed,isCat,Color,ColorPattern,Color2,ColorPattern2,AgeuponOutcome,OutcomeType,SexuponOutcome,Name
0,shetland sheepdog,NaN,True,0,Brown,NaN,White,NaN,365.0,Return_to_owner,Neutered Male,True
1,domestic shorthair,NaN,True,1,Cream,Tabby,NaN,NaN,365.0,Euthanasia,Spayed Female,True
2,pit bull,NaN,True,0,Blue,NaN,White,NaN,730.0,Adoption,Neutered Male,True
3,domestic shorthair,NaN,True,1,Blue,Cream,NaN,NaN,21.0,Transfer,Intact Male,False
4,lhasa apso,miniature poodle,False,0,Tan,NaN,NaN,NaN,730.0,Transfer,Neutered Male,False


In [150]:
test_data.head()

,MixedBreed,Breed,Breed2,isCat,Color,ColorPattern,Color2,ColorPattern2,AgeuponOutcome,OutcomeType,SexuponOutcome,Name
0,True,labrador retriever,NaN,0,Red,NaN,White,NaN,300.0,Adoption,Intact Female,True
1,False,german shepherd,NaN,0,Black,NaN,Tan,NaN,730.0,Adoption,Spayed Female,True
2,True,domestic shorthair,NaN,1,Brown,Tabby,NaN,NaN,365.0,Adoption,Neutered Male,True
3,True,collie smooth,NaN,0,Tricolor,NaN,NaN,NaN,120.0,Adoption,Intact Male,True
4,True,miniature poodle,NaN,0,White,NaN,NaN,NaN,730.0,Adoption,Neutered Male,True


In [155]:
enc = LabelEncoder()
enc_train_data = pd.DataFrame() 
enc_train_data['Breed'] = enc.fit_transform(train_data['Breed'].fillna(''))
enc_train_data['Breed2'] = enc.fit_transform(train_data['Breed2'].fillna(''))
#enc_train_data['MixedBreed'] = enc.fit_transform(train_data['MixedBreed'].fillna(''))
enc_train_data['Color'] = enc.fit_transform(train_data['Color'].fillna(''))
enc_train_data['ColorPattern'] = enc.fit_transform(train_data['ColorPattern'].fillna(''))
enc_train_data['Color2'] = enc.fit_transform(train_data['Color2'].fillna(''))
enc_train_data['ColorPattern2'] = enc.fit_transform(train_data['ColorPattern2'].fillna(''))
#enc_train_data['OutcomeType'] = enc.fit_transform(train_data['OutcomeType'].fillna(''))
enc_train_data['SexuponOutcome'] = enc.fit_transform(train_data['SexuponOutcome'].fillna(''))
#enc_train_data['Name'] = enc.fit_transform(train_data['Name'].fillna(''))
#enc_train_data['AgeuponOutcome'] = enc.fit_transform(train_data['AgeuponOutcome'])

train_data.head()

,Breed,Breed2,MixedBreed,isCat,Color,ColorPattern,Color2,ColorPattern2,AgeuponOutcome,OutcomeType,SexuponOutcome,Name
0,shetland sheepdog,NaN,True,0,Brown,NaN,White,NaN,365.0,Return_to_owner,Neutered Male,True
1,domestic shorthair,NaN,True,1,Cream,Tabby,NaN,NaN,365.0,Euthanasia,Spayed Female,True
2,pit bull,NaN,True,0,Blue,NaN,White,NaN,730.0,Adoption,Neutered Male,True
3,domestic shorthair,NaN,True,1,Blue,Cream,NaN,NaN,21.0,Transfer,Intact Male,False
4,lhasa apso,miniature poodle,False,0,Tan,NaN,NaN,NaN,730.0,Transfer,Neutered Male,False


In [157]:
for col in train_data:
    dtype = train_data.dtypes[col]
    test_data[col]= (test_data[col].astype(dtype))
    print(dtype)
test_data.dtypes

object
object
bool
int64
object
object
object
object
float64
object
object
bool


MixedBreed           bool
Breed              object
Breed2             object
isCat               int64
Color              object
ColorPattern       object
Color2             object
ColorPattern2      object
AgeuponOutcome    float64
OutcomeType        object
SexuponOutcome     object
Name                 bool
dtype: object

In [158]:
test_data.head()

,MixedBreed,Breed,Breed2,isCat,Color,ColorPattern,Color2,ColorPattern2,AgeuponOutcome,OutcomeType,SexuponOutcome,Name
0,True,labrador retriever,NaN,0,Red,NaN,White,NaN,300.0,Adoption,Intact Female,True
1,False,german shepherd,NaN,0,Black,NaN,Tan,NaN,730.0,Adoption,Spayed Female,True
2,True,domestic shorthair,NaN,1,Brown,Tabby,NaN,NaN,365.0,Adoption,Neutered Male,True
3,True,collie smooth,NaN,0,Tricolor,NaN,NaN,NaN,120.0,Adoption,Intact Male,True
4,True,miniature poodle,NaN,0,White,NaN,NaN,NaN,730.0,Adoption,Neutered Male,True


### Some animals have a unknown breed type. Those animals cannot be predicted and thus will be removed. 

In [206]:
trainUniqueBreeds = train_data['Breed'].unique()
for unVal in test_data['Breed'].unique():
    if unVal not in trainUniqueBreeds:
        print(unVal)
        test_data = test_data[test_data.Breed != unVal]

test_data = test_data.reset_index()

In [246]:

enc = LabelEncoder()
enc_train_data = pd.DataFrame() 
enc_test_data = pd.DataFrame() 

enc_train_data['Breed'] = enc.fit_transform(train_data['Breed'].fillna(''))
enc_test_data['Breed'] = enc.transform(test_data['Breed'].fillna(''))

enc_train_data['Breed2'] = enc.fit_transform(train_data['Breed2'].fillna(''))
enc_test_data['Breed2'] = enc.transform(test_data['Breed2'].fillna(''))
#enc_train_data['MixedBreed'] = enc.fit_transform(train_data['MixedBreed'].fillna(''))
enc_train_data['Color'] = enc.fit_transform(train_data['Color'].fillna(''))
enc_test_data['Color'] = enc.transform(test_data['Color'].fillna(''))

enc_train_data['ColorPattern'] = enc.fit_transform(train_data['ColorPattern'].fillna(''))
enc_test_data['ColorPattern'] = enc.transform(test_data['ColorPattern'].fillna(''))

enc_train_data['Color2'] = enc.fit_transform(train_data['Color2'].fillna(''))
enc_test_data['Color2'] = enc.transform(test_data['Color2'].fillna(''))

enc_train_data['ColorPattern2'] = enc.fit_transform(train_data['ColorPattern2'].fillna(''))
enc_test_data['ColorPattern2'] = enc.transform(test_data['ColorPattern2'].fillna(''))

#enc_train_data['OutcomeType'] = enc.fit_transform(train_data['OutcomeType'].fillna(''))
enc_train_data['SexuponOutcome'] = enc.fit_transform(train_data['SexuponOutcome'].fillna(''))
enc_test_data['SexuponOutcome'] = enc.transform(test_data['SexuponOutcome'].fillna(''))
#enc_train_data['Name'] = enc.fit_transform(train_data['Name'].fillna(''))
#enc_train_data['AgeuponOutcome'] = enc.fit_transform(train_data['AgeuponOutcome'])

train_data.head()

,Breed,Breed2,MixedBreed,isCat,Color,ColorPattern,Color2,ColorPattern2,AgeuponOutcome,OutcomeType,SexuponOutcome,Name
0,shetland sheepdog,NaN,True,0,Brown,NaN,White,NaN,365.0,Return_to_owner,Neutered Male,True
1,domestic shorthair,NaN,True,1,Cream,Tabby,NaN,NaN,365.0,Euthanasia,Spayed Female,True
2,pit bull,NaN,True,0,Blue,NaN,White,NaN,730.0,Adoption,Neutered Male,True
3,domestic shorthair,NaN,True,1,Blue,Cream,NaN,NaN,21.0,Transfer,Intact Male,False
4,lhasa apso,miniature poodle,False,0,Tan,NaN,NaN,NaN,730.0,Transfer,Neutered Male,False


In [247]:
temp = test_data['Breed'][test_data['Breed'].isnull()]
temp

Series([], Name: Breed, dtype: object)

In [248]:
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit(enc_train_data)
onehot_encoded =  onehot_encoder.transform(enc_test_data)
print(enc_train_data.shape)
print(onehot_encoded.shape)
print(onehot_encoded)

(26729, 7)
(11442, 440)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


C:\Users\Sjoerd\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [253]:
encodedDataframe = pd.DataFrame(onehot_encoded)

encodedDataframe['MixedBreed'] = test_data['MixedBreed'].astype(int)
encodedDataframe['Name'] = test_data['Name'].astype(int)
encodedDataframe = encodedDataframe.astype(int)

encodedDataframe['AgeuponOutcome'] = test_data['AgeuponOutcome'].fillna(-1)

enc.fit(train_data['OutcomeType'].fillna(''))
encodedDataframe['OutcomeType'] = enc.transform(test_data['OutcomeType'].fillna(''))
encodedDataframe['OutcomeTypeString'] = test_data['OutcomeType'].fillna('')


In [255]:
encodedDataframe.to_csv('../Datasets/test_oneHotEncoded_dataset.csv')


In [254]:
encodedDataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,435,436,437,438,439,MixedBreed,Name,AgeuponOutcome,OutcomeType,OutcomeTypeString
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,1,300.0,0,Adoption
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,730.0,0,Adoption
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,365.0,0,Adoption
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,120.0,0,Adoption
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,1,730.0,0,Adoption
